# AI Competition for the Prediction of Water Level in Hangang River by the Flood Safety Management of Paldang Dam

# Meta data


## Water Data

- ymdhm : 년월일시분
- swl : 팔당댐 현재수위 (단위: El.m)
- inf : 팔당댐 유입량 (단위: m^3/s)
- sfw : 팔당댐 저수량 (단위: 만m^3)
- ecpc : 팔당댐 공용량 (단위: 백만m^3)
- tototf : 총 방류량 (단위: m^3/s)
- tide_level : 강화대교 조위 (단위: cm)
- wl_1018662 : 청담대교 수위 (단위: cm)
- fw_1018662 : 청담대교 유량 (단위: m^3/s)
- wl_1018680 : 잠수교 수위 (단위: cm)
- fw_1018680 : 잠수교 유량 (단위: m^3/s)
- wl_1018683 : 한강대교 수위 (단위: cm)
- fw_1018683 : 한강대교 유량 (단위: m^3/s)
- wl_1019630 : 행주대교 수위 (단위: cm)
- fw_1019630 : 행주대교 유량 (단위: m^3/s)

## RainFall Data

- YMDHM : 년월일시분
- rf_10184100 : 대곡교 강수량
- rf_10184110 : 진관교 강수량
- rf_10184140 : 송정동 강수량
---

## 기타 용어 정리

- 최고 수위 : 일정한 기간을 통하여 나타난 최고의 수위
- 최저 수위 : 일정한 기간을 통하여 나타난 최저의 수위
- 평수위 : 1년을 통하여 185일은 이보다 높은 수위
- 저수위 : 1년을 통하여 275일은 이보다 높은 수위
- 갈수위 : 1년을 통하여 355일은 이보다 높은 수위
- 일평균 수위 : 1일을 통하여 1시부터 24시까지 매시 수위의 합을 24로 나눈 수위
- 연평균 수위 : 1년을 통하여 일평균 수위의 합을 당해 연도의 일수로 나눈 수위
- 감조하천 : 조석의 영향으로 하천의 하류부에서 수위가 변하는 하천


## 주요 링크 모음

한강 홍수 통제소
- http://www.hrfco.go.kr/sumun/rainfallList.do#

한국 수자원 공사
- https://www.kwater.or.kr/main.do?s_mid=1

정보공개
- https://www.open.go.kr/com/main/mainView.do?mainBgGubun=search

In [1]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # 시각화
import seaborn as sns

# 추가 시각화
from colorama import Fore

# MAE MSE
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

# Supress warnings 
import warnings 
warnings.filterwarnings('ignore')

# 재현성
np.random.seed(7)

# 모델들, 성능 평가
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain
from functools import partial

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna
import os

# MICE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# import openpyxl

# flag setting
data_reducing = False ## memory reducing technique
feature_reducing = False ## feature extraction (curse of dimensionality)


from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 대회 metric
def dacon_metric(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    if r2 < 0:
      r2 = 999
    return rmse / r2

evaluation_metric = dacon_metric

## data load

In [2]:
df_water = pd.DataFrame()
df_rf = pd.DataFrame()

# ymdhm을 불러올때부터 Datetime dtype으로 불러오기
for filename in os.listdir('/kaggle/input/dacon-competition-data/water_data'):
    df_water = pd.concat([df_water,pd.read_csv(f'/kaggle/input/dacon-competition-data/water_data/{filename}', parse_dates=['ymdhm'])])
for filename in os.listdir('/kaggle/input/dacon-competition-data/rf_data'):
    df_rf = pd.concat([df_rf,pd.read_csv(f'/kaggle/input/dacon-competition-data/rf_data/{filename}', parse_dates=['ymdhm'])])

df_water = df_water.sort_values(by = ['ymdhm'])
df_rf = df_rf.sort_values(by = ['ymdhm'])
df = pd.concat([df_water,df_rf.drop(columns='ymdhm')], axis = 1,)
df.reset_index(inplace=True, drop =True)

## Missing Values | 결측치 확인

In [3]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,5))

sns.heatmap(df.T.isna(), cmap='Blues')
ax.set_title('Missing Values', fontsize=16)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [4]:
# 결측치가 많고 그나마 있는 데이터도 0인 feature Drop
df.drop(columns=['fw_1018680'], inplace = True)

In [5]:
df.rename(columns={'ymdhm':'date'}, inplace = True)

In [6]:
# test구간인 6월 1일 부터 wl(water level | 수위) feature 값이 모두 0
df.tail(6913)

In [7]:
# MICE sklean 구현
imputed_df = df.copy()
imputed_df = imputed_df.drop(columns='date')

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,
                      verbose = 2, 
                      max_iter = 30,
                      tol = 1e-10,
                      imputation_order='roman')
# fit on the copied dataset
imp.fit(imputed_df)
print('-------- complete fitting -------')
# transform the copied dataset
imputed_df = pd.DataFrame(imp.transform(imputed_df), columns=imputed_df.columns)
imputed_df = pd.concat([df.date, imputed_df], axis=1)

In [8]:
imputed_df.info()

## feature engineering

stationary 분석을 위한 rolling mean, rolling std

In [9]:
# A year has 26469 rows except 2022 year
rolling_window = 26469
f, ax = plt.subplots(nrows=4, ncols=1, figsize=(15, 12))

sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018662'], ax=ax[0], color='dodgerblue')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018662'].rolling(rolling_window).mean(), ax=ax[0], color='black', label='rolling mean')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018662'].rolling(rolling_window).std(), ax=ax[0], color='orange', label='rolling std')
ax[0].set_ylabel(ylabel='wl_1018662', fontsize=14)

sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018680'], ax=ax[1], color='dodgerblue')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018680'].rolling(rolling_window).mean(), ax=ax[1], color='black', label='rolling mean')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018680'].rolling(rolling_window).std(), ax=ax[1], color='orange', label='rolling std')
ax[1].set_ylabel(ylabel='wl_1018680', fontsize=14)

sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018683'], ax=ax[2], color='dodgerblue')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018683'].rolling(rolling_window).mean(), ax=ax[2], color='black', label='rolling mean')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1018683'].rolling(rolling_window).std(), ax=ax[2], color='orange', label='rolling std')
ax[2].set_ylabel(ylabel='wl_1018683', fontsize=14)

sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1019630'], ax=ax[3], color='dodgerblue')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1019630'].rolling(rolling_window).mean(), ax=ax[3], color='black', label='rolling mean')
sns.lineplot(x=imputed_df['date'], y=imputed_df['wl_1019630'].rolling(rolling_window).std(), ax=ax[3], color='orange', label='rolling std')
ax[3].set_ylabel(ylabel='wl_1019630', fontsize=14)

plt.tight_layout()
plt.show()

In [10]:
from statsmodels.tsa.seasonal import seasonal_decompose

core_columns =  [
    'inf','tototf','wl_1018662', 'wl_1018680', 'wl_1018683','wl_1019630'
]
# core_columns = imputed_df.columns.tolist()[1:]
for column in core_columns:
    decomp = seasonal_decompose(imputed_df[column], period=rolling_window, model='additive', extrapolate_trend='freq')
    imputed_df[f"{column}_trend"] = decomp.trend
    imputed_df[f"{column}_seasonal"] = decomp.seasonal

In [11]:
imputed_df.head()

In [12]:
plt.figure(figsize = (14,12))
sns.heatmap(imputed_df.corr(),
            annot = False,
           cmap = "YlGnBu",
           linewidths = "0.5",
           vmin = -1)

In [13]:
# target features
target_col = ['wl_1018662','wl_1018680','wl_1018683','wl_1019630']

In [14]:
X = imputed_df.drop(columns = target_col)
y = pd.concat([imputed_df['date'],imputed_df[target_col]],axis = 1)

In [15]:
# train, test split point
test_split_point = '2022-06-01 00:00:00'

In [16]:
X_train = X[X.date < test_split_point]
y_train = y[y.date < test_split_point]
X_test = X[X.date >= test_split_point]
y_test = y[y.date >= test_split_point]

In [17]:
# 날짜 데이터 ordinal encoding
def create_date_features(df):
    df['year'] = df['date'].dt.year
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['dayofmonth'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['hour'] = df['date'].dt.hour
    df['minute'] = df['date'].dt.minute
    df['dayofyear'] = df['date'].dt.dayofyear    
    df['weekofyear'] = df['date'].dt.weekofyear
    df['date'] = df.index

In [18]:
create_date_features(X_train)
create_date_features(X_test)
y_train = y_train.drop(columns = ['date'])
y_test = y_test.drop(columns = ['date'])

In [19]:
X_train.head()

## Modeling | 모델링

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit


def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    max_depth = trial.suggest_int('max_depth', 0, 16)
    max_leaves = trial.suggest_int('max_leaves', 128, 1023) # 2**max_depth - 1
    n_estimators = trial.suggest_int('n_estimators', 200, 600)
    eta = trial.suggest_uniform('eta', 0.001, 0.3)

    # multi Output regressor
    if y.shape[1] > 1:
        model = RegressorChain(
                    XGBRegressor(n_estimators=n_estimators,
                    max_depth=max_depth,
                    max_leaves=max_leaves,
                    eta=eta,
                    tree_method='gpu_hist' # gpu 사용시
                    # tree_method='hist'
                    ),
                    order=[0, 1, 2, 3])
        model = Pipeline([('reg', model)])
    else:
    # single output regressor
        model = XGBRegressor(n_estimators=n_estimators,
                          max_depth=max_depth,
                          max_leaves=max_leaves,
                          eta=eta,
                          tree_method='gpu_hist', # gpu 사용시
                          # tree_method='hist'
                          n_jobs=-1)
    
    # K-Fold Cross validation
    folds = KFold(n_splits=K)
    losses = []
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        # loss = mean_absolute_error(y_val, preds)
        loss = dacon_metric(y_val, preds)
        losses.append(loss)
#     #Time Series Split (Expanded)
#     folds = TimeSeriesSplit(n_splits=K)
#     losses = []
#     for (train_index, valid_index) in folds.split(X,y):
#         X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
#         y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
#         model.fit(X_train, y_train)
#         preds = model.predict(X_valid)
#         # loss = mean_absolute_error(y_val, preds)
#         loss = dacon_metric(y_valid, preds)
#         losses.append(loss)

    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [21]:
from functools import partial
# Optuna Run!

K = 25 # Kfold 수
study = optuna.create_study(direction="minimize") # 최소/최대 어느 방향의 Loss 최적값을 구할 건지.
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)
study.optimize(opt_func, n_trials=5)

In [22]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [23]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [24]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [25]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

In [26]:
model = RegressorChain(XGBRegressor(**study.best_trial.params, 
                                    tree_method='gpu_hist'
                                    # tree_method='hist'
                                    ), 
                       order=[0, 1, 2, 3]
                       )

In [27]:
model.fit(X_train, y_train)

print("Prediction with Best Estimator")
pred_test = model.predict(X_test)
# np.sqrt(mean_squared_error(y_test, pred_test))
# optuna_test_score = evaluation_metric(y_test, pred_test)
# print("Optuna Test RMSE/R2 Score : %.4f" % optuna_test_score)

# r2_score(y_test, pred_test)

# # save_model(model, 'fitted_model')

In [28]:
submission = pd.read_csv('../input/dacon-competition-data/sample_submission.csv')

for idx, sub_col in enumerate(submission.columns[1:]):
    submission[sub_col] = pred_test[idx]
submission.to_csv('./submission',index=False)

# Thank you